<a href="https://colab.research.google.com/github/RinayGajjar/AI_VoiceAssistant/blob/main/LizMotorsV2T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install whisper transformers pydub sounddevice webrtcvad gtts playsound

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41120 sha256=6a6fced7a7e7e2df31344d03c12c8cb4413d881c71746871b3a9a56391294fd1
  Stored in directory: /root/.cache/pip/wheels/aa/7c/1d/015619716e2facae6631312503baf3c3220e6a9a3508cb14b6
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp310-cp310-linux_x86_64.whl size=73461 sha256=0a203db5a53cb2b6d3d9289edf69eb883b08e112b6bd805fa9c3d9f43dab0c6a
  Stored in directory: /root/.cache/pip/wheels/2a/2b/84/ac7bacfe8c68a87c1ee3dd3c66818a54c71599abf308e8eb35
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=fdc6ab4f27aef52f93c83bde81bf356d508027f6b47328da14ff5f02526cdb0a
  Stored in directory: /ro

In [2]:
from google.colab import files
import shutil

# Upload the audio file from your local system
uploaded = files.upload()

# Move the uploaded file to a specific location
file_name = list(uploaded.keys())[0]
shutil.move(file_name, '/content/testing.m4a')

# Confirm the file has been uploaded and moved
import os
print("Uploaded file:", file_name)
print("File saved at:", os.path.abspath('/content/testing.m4a'))

Saving AudioRec.m4a to AudioRec.m4a
Uploaded file: AudioRec.m4a
File saved at: /content/testing.m4a


In [3]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [7]:
import subprocess

def convert_m4a_to_wav(input_file, output_file):
    command = ['ffmpeg', '-i', input_file, output_file]
    subprocess.run(command, check=True)

# Convert the .m4a file to .wav
convert_m4a_to_wav('/content/AudioRec.m4a', '/content/AudioRec.wav')

In [5]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-vch5oov6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-vch5oov6
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)


In [10]:
import whisper
import openai
import pyttsx3
import webrtcvad

In [9]:
!pip install openai pyttsx3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [11]:
import whisper
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 122MiB/s]


In [12]:
!pip install -q pydub
from pydub import AudioSegment
import io

In [18]:
def process_audio(audio_data):
  audio = AudioSegment.from_file(audio_data)
  audio = audio.set_channels(1)
  audio = audio.set_frame_rate(16000)

  trimmed_audio = audio.strip_silence(silence_thresh=-50,padding=100)
  audio_bytes = io.BytesIO()
  trimmed_audio.export(audio_data, format="wav")
  #audio_data.seek(0)
  return audio_bytes

In [19]:
def transcribe_audio(audio_data):
  audio_bytes = process_audio(audio_data)
  result = model.transcribe(audio_data,language="en")
  return result["text"]

In [20]:
uploaded = files.upload()

Saving AudioRec.m4a to AudioRec (2).m4a


In [21]:
for audio_file_path in uploaded.keys():
    print(f"Processing {audio_file_path}...")
    transcription = transcribe_audio(audio_file_path)
    print("Transcription:", transcription)

Processing AudioRec (2).m4a...


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription:  This is me working on voice to text assignment given by Liz Motors.


In [22]:
!pip install transformers

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [24]:
model_name = "openai-gpt"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/479M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/816k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/458k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

In [27]:
def get_gpt_response(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=80, num_return_sequences=1,temperature = 0.6)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

prompt = "Suggest me a"
response = get_gpt_response(prompt)
print(response)

suggest me a few things. " 
 " like what? " 
 " like how you're a good friend of mine. " 
 " i'm not a friend of yours. " 
 " you're not? " 
 " no. " 
 " then what are you doing here? " 
 " i'm here to help you. " 
 " help me? " 
 " yes. "


In [28]:
!pip install edge-tts

In [29]:
import edge_tts
import asyncio

In [39]:
async def list_voices():
    voices = await edge_tts.list_voices()
    for voice in voices:
        print(f"Voice: {voice['Name']}, Locale: {voice['Locale']}, Gender: {voice['Gender']}")

asyncio.get_event_loop().run_until_complete(list_voices())


Voice: Microsoft Server Speech Text to Speech Voice (af-ZA, AdriNeural), Locale: af-ZA, Gender: Female
Voice: Microsoft Server Speech Text to Speech Voice (af-ZA, WillemNeural), Locale: af-ZA, Gender: Male
Voice: Microsoft Server Speech Text to Speech Voice (sq-AL, AnilaNeural), Locale: sq-AL, Gender: Female
Voice: Microsoft Server Speech Text to Speech Voice (sq-AL, IlirNeural), Locale: sq-AL, Gender: Male
Voice: Microsoft Server Speech Text to Speech Voice (am-ET, AmehaNeural), Locale: am-ET, Gender: Male
Voice: Microsoft Server Speech Text to Speech Voice (am-ET, MekdesNeural), Locale: am-ET, Gender: Female
Voice: Microsoft Server Speech Text to Speech Voice (ar-DZ, AminaNeural), Locale: ar-DZ, Gender: Female
Voice: Microsoft Server Speech Text to Speech Voice (ar-DZ, IsmaelNeural), Locale: ar-DZ, Gender: Male
Voice: Microsoft Server Speech Text to Speech Voice (ar-BH, AliNeural), Locale: ar-BH, Gender: Male
Voice: Microsoft Server Speech Text to Speech Voice (ar-BH, LailaNeural), L

In [65]:
import edge_tts
import asyncio

async def text_to_speech(text, output_file, gender="female"):
    # Determine the voice based on the gender parameter
    voice = "en-US-JennyNeural" if gender == "female" else "en-US-GuyNeural"

    # Initialize the Communicate object with the selected voice
    communicate = edge_tts.Communicate(text, voice=voice)

    # Save the output audio file
    await communicate.save(output_file)

# Example usage
text = "This is me working on a voice assistant for LizMotors."
output_file = "output.wav"

# Run the function with the specified gender (female or male)
asyncio.get_event_loop().run_until_complete(text_to_speech(text, output_file, gender="female"))


In [68]:
!pip install nest_asyncio

In [67]:
import nest_asyncio
nest_asyncio.apply()

In [66]:
import webrtcvad

def remove_silence(audio_data, sample_rate=16000):
    vad = webrtcvad.Vad()
    vad.set_mode(2)  # Set aggressiveness mode (0-3)
    frames = list(audio_data)  # Assume audio_data is a byte array of PCM data
    voiced_frames = [frame for frame in frames if vad.is_speech(frame, sample_rate)]
    return b''.join(voiced_frames)


In [63]:
import nltk
nltk.download('punkt')

def restrict_to_two_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return ' '.join(sentences[:2])

text = "This is the first sentence. This is the second sentence. This is the third sentence."
restricted_text = restrict_to_two_sentences(text)
print(restricted_text)  # Outputs only the first two sentences


This is the first sentence. This is the second sentence.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [69]:
from google.colab import files
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>